<a href="https://colab.research.google.com/github/dearxcorex/ASR_MODEL_NBTC/blob/main/ASR_modal/text_classification/dev_test/data/torchtext.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### NLP from scratch with Pytorch

[Pytorch](https://pytorch.org/tutorials/beginner/text_sentiment_ngrams_tutorial.html)



In [ ]:
!pip install pythainlp deepcut

In [27]:
from pythainlp import word_tokenize
import pandas as pd


# Pytorch
import torch
from torch.utils.data import random_split, DataLoader,Dataset
#from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

In [ ]:
class TweetClassificationDataset(Dataset):
    def __init__(self,csv_file,vocab_size):
        self.data = pd.read_csv(csv_file)
        self.text = self.data["tweet_text"].tolist()
        self.labels = self.data["is_toxic"].tolist()
        self.vocab = build_vocab_from_iterator(self.data['tweet_text'],specials=["<unk>"])
        self.vocab.set_default_index(vocab["<unk>"])

In [6]:
df = pd.read_csv("clean_text.csv", encoding="utf-8")

In [7]:
df.drop("Unnamed: 0", axis=1, inplace=True)

In [8]:
df.head()

,tweet_text,is_toxic
0,วันๆ นี่คุยกะหมา แมว หมู ไก่ ม้า ควาย มากกว่าค...,0
1,หล่อมากกก หล่อวัวตายควายล้มกันเลยทีเดียวววว,0
2,สิวเหี้ยไรขึ้นหลังหู เสียชาติเกิดมั้ย เกิดมามั...,0
3,อ่ะ ป่วยก็ป่วย งานจ้างก็ต้องทำ งานเรียนก็ต้องท...,0
4,นี่ก็เพิ่งรู้ว่า เกิดชาตินี้ชาติเดียว เป็นทั้ง...,1


In [9]:
df['tweet_text'] = df['tweet_text'].apply(lambda x: word_tokenize(x, engine="newmm"))

In [10]:
vocab = build_vocab_from_iterator(df['tweet_text'], specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

In [11]:
vocab(['นาย', 'นาง', 'นางสาว'])

[600, 599, 0]

In [12]:
train_df, test_df = random_split(df, [0.8, 0.2])

AttributeError: 'Subset' object has no attribute 'classes'

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_dataloader = DataLoader(train_df, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_df, batch_size=32, shuffle=True)

### Define model

In [14]:
from torch import nn
class TextClassificationModel(nn.Module):
    def __init__(self, vocab_size,embed_dim,num_class):
        super(TextClassificationModel,self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim,sparse=False)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()


    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [15]:
vocab_size = len(vocab)
emsize = 64
num_class = 2
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)

### dummy test

In [16]:
dummy_text = torch.randint(0, vocab_size, size=(2, 5)).long().to(device)
dummy_offsets = torch.tensor([0, 5]).long().to(device)
model = model.to(device)
print(dummy_text)
print(dummy_offsets)

tensor([[2886, 1441, 1672,  464,  218],
        [4906, 5438, 2143, 1693,  247]], device='cuda:0')
tensor([0, 5], device='cuda:0')


In [17]:
outputs = model(dummy_text,None)
outputs

tensor([[ 0.3852, -0.4360],
        [ 0.1914, -0.0244]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [25]:
%%time
from tqdm.auto import tqdm
#Create a loss function and optimzer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=0.1)

epoch = 5
for epoch in tqdm(range(epoch)):
    train_loss = 0

    for batch,(X,y) in enumerate(train_dataloader):
       model.train()

  0%|          | 0/5 [00:00<?, ?it/s]

KeyError: 1236